# Parsing URL

In [ ]:
import csv
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
from fake_useragent import UserAgent
import urllib
import time
import traceback
import random

In [2]:
#importing the Data set
df=pd.read_csv("combo.csv") 

In [3]:
df.head()

,Unnamed: 0,Recipe ID,Title,Source,Cuisine,Ingredients
0,0,1,5 spice vegetable fried rice,TARLA_DALAL,Indian Subcontinent,"capsicum ,pepper bell ,soy sauce ,sunflower"
1,1,2,aachar aaloo,TARLA_DALAL,Indian Subcontinent,"buttermilk ,cumin ,fenugreek ,ginger garlic pa..."
2,2,3,aadu lassan keri nu athanu,TARLA_DALAL,Indian Subcontinent,"asafoetida ,cayenne ,fenugreek ,ginger garlic ..."
3,3,4,aaloo kofta,TARLA_DALAL,Indian Subcontinent,"butter ,cardamom ,cashew ,cayenne ,cinnamon ,c..."
4,4,5,aaloo tamatar subzi,TARLA_DALAL,Indian Subcontinent,"curry leaf ,lemon ,sunflower"


In [4]:
df.at[0,"Title"]

'5 spice vegetable fried rice'

In [6]:
df.shape

(45772, 6)

In [6]:
#function for delaying 5 second after each parsing to avoid getting ban
def func(m):                  
        time.sleep(5)

In [7]:
#creating a list to store the links
all_links=[""]*45772

In [22]:
#parsing the URLS
for m in range(len(df)):
    #s is the string you want to search on Google
    s=df.at[m,"Title"]+" "+df.at[m,"Source"]  
    s=s.replace("_"," ")
    query = s
    query = urllib.parse.quote_plus(query) # Format into URL encoding
    number_result = 20

    ua = UserAgent()

    google_url = "https://www.google.com/search?q=" + query + "&num=" + str(number_result)
    response = requests.get(google_url,{"User-Agent": ua.random})           
    soup = BeautifulSoup(response.text, "html.parser")
    #print(soup)
    result_div = soup.find_all('div', attrs = {'class': 'ZINbbc'})
   
    links = []
    titles = []
    descriptions = []
    for r in result_div:
        # Checks if each element is present, else, raise exception
        try:
            link = r.find('a', href = True)
            title = r.find('div', attrs={'class':'vvjwJb'}).get_text()
            description = r.find('div', attrs={'class':'s3v9rd'}).get_text()

            # Check to make sure everything is present before appending
            if link != '' and title != '' and description != '': 
                links.append(link['href'])
                titles.append(title)
                descriptions.append(description)
        # Next loop if one element is not present
        except:
            continue
    to_remove = []
    clean_links = []
    for i, l in enumerate(links):
        clean = re.search('\/url\?q\=(.*)\&sa',l)

        # Anything that doesn't fit the above pattern will be removed
        if clean is None:
            to_remove.append(i)
            continue
        clean_links.append(clean.group(1))

    # Remove the corresponding titles & descriptions 
    func(m)
    if(len(clean_links)>0):
        all_links[m]=clean_links[0]
    else:
        print(m)
        break
    if(m%10==0):
        print(m,len(clean_links))

0 18
10 19
20 20
30 19
40 19
50 19
60 19
70 19
80 19
90 19
100 18
110 19
120 18
130 18
140 20
150 20
160 19
170 19
180 19
190 18
200 20
210 19
220 19
230 18
240 20
250 19
260 19
270 20
280 19
290 19
300 19
310 18
320 18
330 19
340 20
350 19
360 19
370 19
380 19
390 12
400 19


KeyboardInterrupt: 

In [13]:
all_links[30000:30002]

['https://www.foodnetwork.com/recipes/steamboat-natchez-sauteed-shrimp-recipe-1915206',
 'https://www.foodnetwork.com/recipes/barmouches-picnic-coleslaw-recipe-1915125']

In [23]:
#adding a 'links' column in the dataframe
df['Links']=all_links

In [24]:
#storing the dataframe in a csv file
df.to_csv('lala.csv')